In [26]:
import numpy as np
import pandas as pd
from influxdb_client import InfluxDBClient, Point
from influxdb_client.client.write_api import SYNCHRONOUS

import warnings
from influxdb_client.client.warnings import MissingPivotFunction
#warnings.simplefilter("ignore", MissingPivotFunction)


client = InfluxDBClient(url="http://192.168.0.112:8086", token='appdaemon-dev:opennow', org='-')
query_api = client.query_api()

def get_sensor_data(entity_id, column, start='-7d'):
    id_parts = entity_id.split('.')
    if len(id_parts) == 2:
        id_parts.append("value") # default for state field, otherwise attr
    q = f'''
    from(bucket: "homeassistant/autogen")
      |> range(start: {start})
      |> filter(fn: (r) => r.entity_id == "{id_parts[1]}" and r._field == "{id_parts[2]}" and r.domain == "{id_parts[0]}")
      |> pivot(rowKey:["_time"], columnKey: ["entity_id"], valueColumn: "_value")
      |> drop(columns: ["_measurement", "domain", "_start", "_stop"])
    '''
#     print(q)
    df = query_api.query_data_frame(q).drop(['result','table','_field'], axis=1)
    return df.rename(columns={id_parts[1]: column})

def compute_heat_index(temp_farenheit, relative_humidity):
    # relative humidity should be 0-100
    if temp_farenheit < 80: # seems like the formula below says it doesn't apply below 80ºF
        return temp_farenheit
    t = temp_farenheit
    rh = relative_humidity
    # https://www.weather.gov/media/epz/wxcalc/heatIndex.pdf
    return -42.379 + 2.04901523 * t + 10.14333127 * rh - 0.22475541 * t * rh - 6.83783e-3 * t * t - 5.481717e-2 * rh * rh + 1.22874e-3 * t * t * rh + 8.5282e-4 * t * rh * rh - 1.99e-6 * t * t * rh * rh

In [18]:
remote_sensor = 'bedroom'
remote_sensor = 'main_floor'
# remote_sensor = 'aysylu_office'

In [3]:
humidity = get_sensor_data(f"{remote_sensor}_sensor_bme280_humidity") # what we want to calibrate to
remote_temp = get_sensor_data(f"{remote_sensor}_sensor_bme280_temperature") # what we want to calibrate to
temp = get_sensor_data("thermostat_2", field="current_temperature") # thermostat's opinion
target_temp= get_sensor_data("thermostat_2", field="temperature") # thermostat's setting
mode = get_sensor_data("thermostat_2", field="hvac_action_str") # fan, heating, or cooling
hi = f"heat index of 80ºF at 90%rh = {compute_heat_index(80, 90)}"


# First, join all data together with the fill forward or something
# Add columns for feels like for the temperatures
## To find offset
# Filter for rows where the thermostat is not in "off" or "fan" mode
# Filter for rows where the thermostat thinks it hit its target
# Group by heat & cool modes to get the heat & cool fixed offsets
## To find speed of action (º/min or something)
# Using shift to find changes, filter for rows when the mode or target changed AND mode is heating or cooling
# Also filter for rows when the target temp was successfully hit
# Join those 2 frames together the initiated change & arrival time pairing
# Group by mode & compute averages (maybe confirm distributions are reasonable)

#print(temp)
#print(humidity)
base_times = pd.concat([x['_time'] for x in [temp, humidity, remote_temp, target_temp, mode]])
base_times = base_times.sort_values().drop_duplicates()
df = pd.merge_asof(base_times, temp, on='_time')
df = pd.merge_asof(df, target_temp, on='_time')
df = pd.merge_asof(df, humidity, on='_time')
df = pd.merge_asof(df, mode, on='_time')
df = pd.merge_asof(df, remote_temp, on='_time')
offset_df = df.query("hvac_action_str in ('heating', 'cooling') and current_temperature == temperature").copy()
offset_df['_time'] = offset_df['_time'].dt.tz_convert('America/New_York')
offset_df['delta'] = offset_df[f'{remote_sensor}_sensor_bme280_temperature'] - offset_df['current_temperature']
# #print(offset_df)
# #print(offset_df[['hvac_action_str', 'delta']].groupby('hvac_action_str').describe())
df = df.set_index('_time')


#print(change_events_df)

# Then, we'll have hot & cold offsets and hot & cold speeds
# These values may need to be additionally bucketed/broken down by time of day, difference between indoor/outdoor temp, or weather

In [4]:
offset_df

,_time,current_temperature,temperature,main_floor_sensor_bme280_humidity,hvac_action_str,main_floor_sensor_bme280_temperature,delta
832,2022-10-09 04:53:57.534962-04:00,68.0,68.0,42.9,heating,68.4,0.4
833,2022-10-09 04:54:42.980894-04:00,68.0,68.0,42.9,heating,68.5,0.5
834,2022-10-09 04:54:42.996007-04:00,68.0,68.0,42.8,heating,68.5,0.5
835,2022-10-09 04:55:42.981232-04:00,68.0,68.0,42.8,heating,68.7,0.7
836,2022-10-09 04:55:42.996714-04:00,68.0,68.0,42.6,heating,68.7,0.7
...,...,...,...,...,...,...,...
2796,2022-10-11 08:38:40.668897-04:00,71.0,71.0,43.8,heating,72.5,1.5
2797,2022-10-11 08:39:40.678522-04:00,71.0,71.0,43.8,heating,72.3,1.3
2798,2022-10-11 08:43:40.670277-04:00,71.0,71.0,43.8,heating,72.5,1.5
2799,2022-10-11 08:43:40.689625-04:00,71.0,71.0,43.7,heating,72.5,1.5


In [5]:
offset_df[['hvac_action_str', 'delta']].groupby('hvac_action_str').describe()

delta                                             
                 count      mean       std  min  25%  50%  75%  max
hvac_action_str                                                    
cooling           93.0 -1.503226  0.846791 -2.7 -2.3 -1.8 -0.7  0.0
heating          243.0  0.999177  0.538286 -0.7  0.7  1.1  1.3  2.2

In [85]:
df = df.drop([f'{remote_sensor}_sensor_bme280_temperature', f'{remote_sensor}_sensor_bme280_humidity'],axis=1)

In [86]:
pd.merge(df, change_df, left_index=True, right_index=True).dropna(subset=['hvac_action_str', 'action_before']).query("(hvac_action_str != action_before or temperature != target_before)")

,current_temperature,temperature,hvac_action_str,action_before,target_before
_time,,,,,
2022-10-09 08:53:57.534962+00:00,68.0,68.0,heating,fan,68.0
2022-10-09 08:58:57.614072+00:00,68.0,68.0,fan,heating,68.0
2022-10-09 09:33:57.785094+00:00,68.0,68.0,heating,fan,68.0
2022-10-09 09:43:57.703849+00:00,68.0,68.0,fan,heating,68.0
2022-10-09 10:03:57.678314+00:00,68.0,68.0,heating,fan,68.0
2022-10-09 11:23:57.846716+00:00,68.0,68.0,fan,heating,68.0
2022-10-09 11:48:57.771224+00:00,68.0,68.0,heating,fan,68.0
2022-10-09 11:53:57.887257+00:00,68.0,68.0,fan,heating,68.0
2022-10-10 02:13:59.118018+00:00,68.0,68.0,heating,fan,68.0


In [87]:
change_df = df[['hvac_action_str', 'temperature']].shift(1).rename(columns={'hvac_action_str': 'action_before', 'temperature': 'target_before'})
change_events_df = pd.merge(df, change_df, left_index=True, right_index=True).dropna(subset=['hvac_action_str', 'action_before']).query("(hvac_action_str != action_before or temperature != target_before) and hvac_action_str not in ['fan', 'off'] and current_temperature != temperature")

stable_df = df.query('current_temperature == temperature').drop(['current_temperature', 'temperature'],axis=1)
stable_df['stable_time'] = stable_df.index
change_events_df['temp_delta'] = change_events_df['temperature'] - change_events_df['current_temperature']
change_events_df = pd.merge_asof(change_events_df, stable_df, left_index=True, right_index=True, by='hvac_action_str', direction='forward').dropna(subset=['stable_time'])
change_events_df['time_delta'] = (change_events_df['stable_time'] - change_events_df.index).dt.total_seconds()
change_events_df['adapt_rate_degrees_per_hr'] = change_events_df['temp_delta'] / (change_events_df['time_delta'] / 3600.0)
change_events_df

,current_temperature,temperature,hvac_action_str,action_before,target_before,temp_delta,stable_time,time_delta,adapt_rate_degrees_per_hr
_time,,,,,,,,,
2022-10-10 18:24:00.922272+00:00,75.0,73.0,cooling,fan,69.0,-2.0,2022-10-10 19:09:01.199282+00:00,2700.27701,-2.666393


In [88]:
"foo".startswith('f')

True

In [30]:
remote_temp_ent = 'sensor.main_floor_sensor_bme280_temperature'
thermostat_ent='climate.thermostat_2'
remote_temp = get_sensor_data(remote_temp_ent, column='remote_temp')
temp = get_sensor_data(thermostat_ent + ".current_temperature", column='current_temp')
target_temp = get_sensor_data(thermostat_ent + ".temperature", column='target_temp')
mode = get_sensor_data(thermostat_ent + ".state", column='hvac_action')
base_times = pd.concat([x['_time'] for x in [temp, remote_temp, target_temp, mode]])
base_times = base_times.sort_values().drop_duplicates()
df = pd.merge_asof(base_times, temp, on='_time')
df = pd.merge_asof(df, target_temp, on='_time')
df = pd.merge_asof(df, mode, on='_time')
df = pd.merge_asof(df, remote_temp, on='_time')
df
offset_df = df.query("hvac_action in ('heat', 'cool') and current_temp == target_temp").copy()
offset_df._time= offset_df._time.dt.tz_convert('America/New_York')
offset_df['delta'] = offset_df['remote_temp'] - offset_df['current_temp']
offset_df
result = offset_df[['hvac_action', 'delta']].groupby('hvac_action').describe()
result

delta                                             
             count      mean       std  min  25%  50%  75%  max
hvac_action                                                    
cool         276.0  0.220652  0.948975 -2.7 -0.2  0.3  0.9  2.1
heat         271.0  1.039483  0.776668 -0.7  0.5  0.9  1.5  3.6

In [117]:
Out[110].loc['cooling'].loc[('delta','mean')]

-1.3583333333333332

In [50]:
temp = get_sensor_data(thermostat_ent + ".current_temperature", column="current_temp")
target_temp = get_sensor_data(thermostat_ent + ".temperature", column="target_temp")
mode = get_sensor_data(thermostat_ent + ".state", column="hvac_mode")
base_times = pd.concat([x['_time'] for x in [temp, target_temp, mode]])
base_times = base_times.sort_values().drop_duplicates()
df = pd.merge_asof(base_times, temp, on='_time')
df = pd.merge_asof(df, target_temp, on='_time')
df = pd.merge_asof(df, mode, on='_time')
df['_time'] = df['_time'].dt.tz_convert('America/New_York')
df = df.set_index('_time')
change_df = df[['hvac_mode', 'target_temp']].shift(1).rename(columns={'hvac_mode': 'mode_before', 'target_temp': 'target_before'})
change_events_df = pd.merge(df, change_df, left_index=True, right_index=True).dropna(subset=['hvac_mode', 'mode_before'])
change_events_df = change_events_df.query("(hvac_mode != mode_before or target_temp != target_before) and hvac_mode not in ['fan', 'off'] and current_temp != target_temp")
change_events_df


stable_df = df.query('current_temp == target_temp').drop(['current_temp'],axis=1).rename(columns={'target_temp': 'stable_temp'})
stable_df['stable_time'] = stable_df.index
change_events_df['temp_delta'] = change_events_df['target_temp'] - change_events_df['current_temp']
change_events_df = pd.merge_asof(change_events_df, stable_df, left_index=True, right_index=True, by='hvac_mode', direction='forward').dropna(subset=['stable_time'])
change_events_df = change_events_df.query("(temp_delta > 0 and hvac_mode == 'heat') or (temp_delta < 0 and hvac_mode == 'cool')")
change_events_df['time_delta'] = (change_events_df['stable_time'] - change_events_df.index).dt.total_seconds()
change_events_df['adapt_rate_degrees_per_hr'] = change_events_df['temp_delta'] / (change_events_df['time_delta'] / 3600.0)
change_events_df



,current_temp,target_temp,hvac_mode,mode_before,target_before,temp_delta,stable_temp,stable_time,time_delta,adapt_rate_degrees_per_hr
_time,,,,,,,,,,
2022-10-10 14:24:00.922272-04:00,75.0,73.0,cool,heat,69.0,-2.0,73.0,2022-10-10 15:09:01.199282-04:00,2700.277010,-2.666393
2022-10-11 06:19:02.623873-04:00,69.0,71.0,heat,heat,69.0,2.0,71.0,2022-10-11 06:22:03.769376-04:00,181.145503,39.747054
2022-10-13 20:45:35.470213-04:00,72.0,71.0,cool,cool,72.0,-1.0,70.0,2022-10-13 20:50:35.274542-04:00,299.804329,-12.007832
2022-10-13 20:45:39.855029-04:00,72.0,70.0,cool,cool,71.0,-2.0,70.0,2022-10-13 20:50:35.274542-04:00,295.419513,-24.372121
